## Results
Fit models to real-world data, as shown in @fig-fit.

In [ ]:
import warnings
from typing import List
import numpy as np
import pandas as pd
from datetime import datetime
import nevergrad as ng
from IPython.display import Markdown
from collections import namedtuple

from emu_renewal.renew import RenewalModel, TruncRenewalModel
from emu_renewal.outputs import plot_output_fit

In [ ]:
# Data and fixed parameters
run_in = 30
n_process_periods = 12
raw_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)['MYS']
raw_data.index = pd.to_datetime(raw_data.index)
mys_data = raw_data.loc[datetime(2021, 3, 1): datetime(2021, 11, 1)].reset_index()['MYS']
mys_data.index += run_in
n_times = len(mys_data) + run_in
calib_kwargs = {'pop': 33e6, 'n_times': n_times, 'run_in': run_in, 'targets': mys_data}
fixed_param_desc = '### Fixed parameter values\n ' \
    f'The target population is initialised as {str(int(calib_kwargs["pop"]))} susceptible persons. ' \
    f'The simulation runs for a run-in period of {run_in} days before comparison against the calibration data commences.\n'

In [ ]:
# Define parameter ranges
scalar_req = {
    'Generation time mean (days)' : {'init': 5.0, 'lower': 0.1, 'upper': 14.0},
    'Generation time standard deviation (days)': {'init': 5.0, 'lower': 2.5, 'upper': 8.0},
    'Case detection proportion': {'init': 0.06, 'lower': 0.04, 'upper': 0.2},
    'Log starting seed rate': {'init': np.log(1e4), 'lower': np.log(5e3), 'upper': np.log(2e4)},
}
params_df = pd.DataFrame(scalar_req).transpose()
proc_req = {'init': 0.0, 'lower': -2.0, 'upper': 2.0}

In [ ]:
#| warning: false
calib_desc = '\n\n### Calibration targets\nThe model described above was fit to the target data ' \
    'to minimise the square of the difference between the modelled notification rate ' \
    'and the reported case numbers at each date considered. ' \
    'Modelled notifications are calculated as the product of modelled incidence and the ' \
    '(constant through time) case detection proportion. '

model = RenewalModel(calib_kwargs['pop'], n_times, run_in, n_process_periods)

def calib_func(parameters: List[float], pop: int, n_times: int, run_in: int, targets: dict) -> float:
    gen_time_mean, gen_time_sd, cdr, seed, *process = parameters
    incidence = model.func(gen_time_mean, gen_time_sd, process, seed).incidence
    return sum([(incidence[t] * cdr - d) ** 2 for t, d in targets.items()])

def obj_func(gen_time_mean, gen_time_sd, cdr_param, seed, proc_params):
    return calib_func([gen_time_mean, gen_time_sd, cdr_param, seed] + list(proc_params), **calib_kwargs)

scalar_params = [ng.p.Scalar(**row).set_name(idx) for idx, row in params_df.iterrows()]
process_params = ng.p.Array(init=[proc_req['init']] * n_process_periods, lower=proc_req['lower'], upper=proc_req['upper']).set_name('Log non-mechanistic process values')
instrum = ng.p.Instrumentation(*scalar_params, process_params)
optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=100)
ngopt_result = optimizer.minimize(obj_func).value[0]

In [ ]:
#| label: fig-fit
#| fig-cap: "Optimisation to sample data from Malaysia"
model_result = model.func(ngopt_result[0], ngopt_result[1], ngopt_result[4], ngopt_result[3])
fig = plot_output_fit(mys_data, model_result, n_times, cdr=ngopt_result[2])
fig.write_image('results_fig.svg')

![Results of fitting to data from Malaysia.](results_fig.svg){#fig-fit}

## Methods

In [ ]:
Markdown(fixed_param_desc) 

In [ ]:
Markdown(model.get_description())

In [ ]:
Markdown(calib_desc)

### Calibrated parameters
The parameters used in the optimisation presented here are presented in @tbl-params.

In [ ]:
params_df.loc['Log non-mechanistic process values'] = proc_req
params_df.columns = ['Starting point', 'Lower limit', 'Upper limit']
Markdown(params_df.to_markdown() + '\n : Calibration parameters table {#tbl-params}')

In [ ]:
evidence_table = pd.DataFrame(index=params_df.index, columns=['Evidence'])
evidence_table.loc[:, 'Evidence'] = 'To be populated [@cori2013]'
Markdown(evidence_table.to_markdown())

## References